## 数据归一化
清除一些用来展示的代码，清空之前的运行结果

### 数据读取与展示

In [1]:
# 使用tf.keras搭建分类模型，数据集使用fashion_mnist
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
#导入数据集并拆分数据集
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_train.shape, y_valid.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(55000, 28, 28) (5000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)


### 增加代码归一化

In [3]:
# 归一化方法：x = (x - u) / std 均值，方差

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None, 28, 28] -> [None, 784]
# 除了transform功能还有fit的功能（记住训练集的均值方差），验证集和测试集需要使用训练集的均值和方差
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

### 模型构建
不同版本API，代码不尽相同

In [5]:
# 使用tf.keras.model.Sequential()构建模型

# 创建Sequential对象
model = keras.models.Sequential()
# 添加输入层,将输入图片展平，将28X28的矩阵展开为一维向量
model.add(keras.layers.Flatten(input_shape=[28, 28]))
# 添加全连接层，神经元设为300，激活函数为‘relu’
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
# 添加输出，长度为10的向量，激活函数为'softmax'
model.add(keras.layers.Dense(10, activation='softmax'))

# 另外一种写法
#model = keras.models.Sequential([
#    keras.layers.Flatten(input_shape=[28,28]),
#    keras.layers.Dense(300, activation='relu'),
#    keras.layers.Dense(100, activation='relu'),
#    keras.layers.Dense(10, activation='softmax')
#])

# relu: y=max(0,x)
# softmax: 将向量变成概率分布，x = [x1, x2, x3],三个数加起来为1
#          y = [e^x1/sum, e^x2/sum, e^x3/sum], sum=e^x1+e^x2+e^x3

# 计算目标函数
# 损失函数使用'sparse_categorical_crossentropy'，即交叉熵，categorical类别
# reason for sparse: y_valid是长度等于样本数目的一个向量，y只是一个index值使用'sparse_categorical_crossentropy'
# y->one hot->[]将y转换为向量使用'categorical_crossentropy'
# 2.0版本可以直接使用optimizer='sgd',学习率的默认值不同
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='sgd', metrics=['accuracy'])

### 增加回调函数

In [16]:
# 开启训练fit函数
# epochs训练次数，history返回为中间运算的一些结果
# 回调函数fit函数中添加数组
# Tensorboard,earlystopping,ModelCheckpoint
#Tensorboard需要一个文件夹,ModelCheckpoint需要一个文件名
#在当前文件夹下定义一个名为callbacks的文件夹，如果不存在，便创建它
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
# 定义输出model的文件,也是在callbacks文件夹下
output_model_file = os.path.join(logdir,
                                'fashion_mnist_model.h5')
callbacks = [
    #keras.callbacks.TensorBoard(logdir),
    # 保存最好的模型，否则默认保存最近的模型
    keras.callbacks.ModelCheckpoint(output_model_file,
                                   save_best_onlly=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]

history = model.fit(x_train_scaled, y_train, epochs=10,
         validation_data=(x_valid_scaled,y_valid),
                   callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 19s 346us/sample - loss: 0.1012 - accuracy: 0.9653 - val_loss: 0.3205 - val_accuracy: 0.8968
Epoch 2/10
55000/55000 [==============================] - 18s 323us/sample - loss: 0.0983 - accuracy: 0.9664 - val_loss: 0.3516 - val_accuracy: 0.8898
Epoch 3/10
55000/55000 [==============================] - 19s 341us/sample - loss: 0.0942 - accuracy: 0.9677 - val_loss: 0.3365 - val_accuracy: 0.8910
Epoch 4/10
55000/55000 [==============================] - 23s 417us/sample - loss: 0.0909 - accuracy: 0.9690 - val_loss: 0.3261 - val_accuracy: 0.8970
Epoch 5/10
55000/55000 [==============================] - 28s 504us/sample - loss: 0.0877 - accuracy: 0.9698 - val_loss: 0.3334 - val_accuracy: 0.8974
Epoch 6/10
55000/55000 [==============================] - 28s 506us/sample - loss: 0.0842 - accuracy: 0.9713 - val_loss: 0.3286 - val_accuracy: 0.8992


查看tensorboard，在命令行中输入tensorboard --logdir=callbacks,构建一个个人服务器，通过6006端口进行访问，在浏览器中输入localhost:6006查看tensorfboard

In [ ]:
# 将history中的值可视化出来
def plot_learning_curves(history):
    #直接转换为DataFrame
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    # 显示网格
    plt.grid(True)
    # 设置坐标轴范围
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test)